IWDGD中小值的确定性噪声 对应到DGD算法中是固定的对偶攻击值为-18.6 这样两个算法各自迭代2000次 所有用户的对偶迭代值均在-18.57附近

鲁棒聚合器CTM 每个用户剔除3个用户对偶信息（包含自身的信息）3-是网络中噪声用户数量的上界

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(54,2000)) 
dual_variable = np.zeros(shape=(54,2000))#对偶变量
sum_resource=np.zeros(shape=(1,2000), dtype=np.float64)
sum_cost=np.zeros(shape=(1,2000), dtype=np.float64)
iteration_update = 0.01 #迭代步长
#W=np.zeros(shape=(54,54)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
W=np.loadtxt("double_stochastic_weight_1.txt") #双随机的网络拓扑矩阵W（对角线处非零）
iteration=[]#绘图横坐标
generator_a_values= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
A=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑邻接矩阵（对角线处为0）
D=np.loadtxt("degree_generator.txt") #用户的度
neighbor=np.zeros(shape=(54,17))#定义矩阵存放54个用户的邻居
neighbor_help=[]#辅助求用户邻居的列表

In [3]:
#对偶变量的初始化 初始化一致
for i in range (0,54):
    dual_variable[i][0]=0
    
#资源量的初始化 资源量的初始化需要和对偶变量的初始值满足某种关系
for i in range (0,54):
    agent_resource_quantity[i][0]=6000/54

iteration.append(0)

#总资源量 初始化
sum=0
for i in range (0,54):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

#用户总成本初始化
sum=0
for i in range(0,54):
    sum=sum+(generator_c_values[i]*(agent_resource_quantity[i][0])*(agent_resource_quantity[i][0])+generator_b_values[i]*agent_resource_quantity[i][0]+generator_a_values[i])
sum_cost[0][0]=sum

#对用户邻居进行初始化
for i in range(0,54):
    for j in range(0,17):
        neighbor[i][j]=99 #先将矩阵中的所有数字设定为一个不可能的数字
#根据邻接矩阵求每个用户的邻居集合
for i in range(0,54):
    for j in range(0,54):
        if A[i][j]!=0:
            neighbor_help.append(j)
l=0
for i in range(0,54):
    g=0
    for j in range(l,l+int(D[i])):
        neighbor[i][g]=neighbor_help[j]  
        g=g+1
    l=l+int(D[i])    

In [4]:
#DGD算法 
def DGD_CTM_determinitic_bias(D,neighbor,generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,dual_variable,iteration_update,W,iteration):
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+无计算误差****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    for i in range(0,54):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        iteration.append(k)
        
        #所有用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate=[] #存放当前迭代中54个用户对对偶变量做完梯度下降后得到的中间值
        for i in range(0,54):
            #每个用户对对偶变量先进行一次梯度下降
            if i==48:
                dual_variable_intermediate.append(-18.6)
            else:
                dual_variable_intermediate.append(dual_variable[i][k-1]-iteration_update*((6000/54)-agent_resource_quantity[i][k-1]))             
        #阶段二：鲁棒聚合 CTM       
        for i in range(0,54):
            #先根据用户的度来判断其是否需要做CTM操作 
            if int(D[i])>3: 
                u_neighbor_help=[]
                sort_list=[] 
                mix_sum_dual=0
                for j in range(0,int(D[i])):
                    u_neighbor_help.append(neighbor[i][j])                               
                for j in range(0,int(D[i])):
                    sort_list.append(dual_variable_intermediate[int(u_neighbor_help[j])])
                sort_list.append(dual_variable_intermediate[i])
                sort_list.sort()
                #剔除一个最大的数据和一个最小的数据  
                del sort_list[-1]
                del sort_list[0]
                #剩下的数据进行平均操作     
                #用户基于TM鲁棒聚合后的值进行自身的对偶变量的更新 平均值
                for j in range(0,len(sort_list)):
                    mix_sum_dual=mix_sum_dual+sort_list[j]                      
                dual_variable[i][k]=(1/(len(sort_list)))*mix_sum_dual
            else:
                mix_sum_dual=0           
                for j in range(0,54):
                    mix_sum_dual=mix_sum_dual+W[i][j]*dual_variable_intermediate[j]
                dual_variable[i][k]=mix_sum_dual                    
        
        #用户资源申请量的迭代
        for i in range (0,54):
            agent_resource_quantity[i][k]=(-dual_variable[i][k]-generator_b_values[i])/(2*generator_c_values[i]) 

        #计算所有用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,54):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有用户的成本总和
        sum2=np.float64(0)
        for i in range(0,54):
            sum2=sum2+(generator_a_values[i]+(generator_b_values[i]*agent_resource_quantity[i][k])+(generator_c_values[i]*agent_resource_quantity[i][k]*agent_resource_quantity[i][k]))
        sum_cost[0][k]=sum2
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%20==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,54):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            for i in range(0,54):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=(1/(pow(k+1,1.3)))*(1/26.511) #递减步长
        iteration_update=(1/(pow(k+1,1.5)))*(1/18.8468) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
DGD_CTM_determinitic_bias(D,neighbor,generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,dual_variable,iteration_update,W,iteration)

****************************DGD算法+无计算误差****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 111.11111111111111
agent 2 的初始资源申请量为: 111.11111111111111
agent 3 的初始资源申请量为: 111.11111111111111
agent 4 的初始资源申请量为: 111.11111111111111
agent 5 的初始资源申请量为: 111.11111111111111
agent 6 的初始资源申请量为: 111.11111111111111
agent 7 的初始资源申请量为: 111.11111111111111
agent 8 的初始资源申请量为: 111.11111111111111
agent 9 的初始资源申请量为: 111.11111111111111
agent 10 的初始资源申请量为: 111.11111111111111
agent 11 的初始资源申请量为: 111.11111111111111
agent 12 的初始资源申请量为: 111.11111111111111
agent 13 的初始资源申请量为: 111.11111111111111
agent 14 的初始资源申请量为: 111.11111111111111
agent 15 的初始资源申请量为: 111.11111111111111
agent 16 的初始资源申请量为: 111.11111111111111
agent 17 的初始资源申请量为: 111.11111111111111
agent 18 的初始资源申请量为: 111.11111111111111
agent 19 的初始资源申请量为: 111.11111111111111
agent 20 的初始资源申请量为: 111.11111111111111
agent 21 的初始资源申请量为: 111.11111111111111
agent 22 的初始资源申请量为: 111.11111111111111
agent 23 的初始资源申请量为: 111.11111111111111
agent 24 的初始资源申请量为: 111.1111111111

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 54.716687710748936
agent 2 在 3999 次迭代中的资源申请量为: 54.567522229843206
agent 3 在 3999 次迭代中的资源申请量为: 59.023900053580434
agent 4 在 3999 次迭代中的资源申请量为: 36.92975132697704
agent 5 在 3999 次迭代中的资源申请量为: 56.0518202139615
agent 6 在 3999 次迭代中的资源申请量为: 53.888978176820906
agent 7 在 3999 次迭代中的资源申请量为: 63.85317569252553
agent 8 在 3999 次迭代中的资源申请量为: 45.1130480444457
agent 9 在 3999 次迭代中的资源申请量为: 63.89345117272944
agent 10 在 3999 次迭代中的资源申请量为: 52.80415365954715
agent 11 在 3999 次迭代中的资源申请量为: 63.66911111803924
agent 12 在 3999 次迭代中的资源申请量为: 37.39676155531569
agent 13 在 3999 次迭代中的资源申请量为: 48.35974451605358
agent 14 在 3999 次迭代中的资源申请量为: 40.28862991046608
agent 15 在 3999 次迭代中的资源申请量为: 41.37743136177737
agent 16 在 3999 次迭代中的资源申请量为: 47.95009974051949
agent 17 在 3999 次迭代中的资源申请量为: 48.81123819513839
agent 18 在 3999 次迭代中的资源申请量为: 45.31079367193395
agent 19 在 3999 次迭代中的资源申请量为: 43.57485640126277
agent 20 在 3999 次迭代中的资源申请量为: 49.479032413134824
agent 21 在 3999 次迭代中的资源申请量为: 68.07505041550442
agent

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 55.82491565839552
agent 2 在 5999 次迭代中的资源申请量为: 55.66757890926747
agent 3 在 5999 次迭代中的资源申请量为: 60.23826433123413
agent 4 在 5999 次迭代中的资源申请量为: 37.68016362132643
agent 5 在 5999 次迭代中的资源申请量为: 57.18472242892364
agent 6 在 5999 次迭代中的资源申请量为: 54.97554537347114
agent 7 在 5999 次迭代中的资源申请量为: 65.17702224789741
agent 8 在 5999 次迭代中的资源申请量为: 46.02371065631031
agent 9 在 5999 次迭代中的资源申请量为: 65.20141246896623
agent 10 在 5999 次迭代中的资源申请量为: 53.87714209301658
agent 11 在 5999 次迭代中的资源申请量为: 65.00226812533164
agent 12 在 5999 次迭代中的资源申请量为: 38.15156835230488
agent 13 在 5999 次迭代中的资源申请量为: 49.36233634998287
agent 14 在 5999 次迭代中的资源申请量为: 41.09420765804113
agent 15 在 5999 次迭代中的资源申请量为: 42.20824313501914
agent 16 在 5999 次迭代中的资源申请量为: 48.91856428464321
agent 17 在 5999 次迭代中的资源申请量为: 49.788740776338074
agent 18 在 5999 次迭代中的资源申请量为: 46.22789059162901
agent 19 在 5999 次迭代中的资源申请量为: 44.47143659459913
agent 20 在 5999 次迭代中的资源申请量为: 50.46930282631791
agent 21 在 5999 次迭代中的资源申请量为: 69.4649555538241
agent 22

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.09553066888908
agent 2 在 7999 次迭代中的资源申请量为: 55.93619860116216
agent 3 在 7999 次迭代中的资源申请量为: 60.534796468361805
agent 4 在 7999 次迭代中的资源申请量为: 37.86340465160463
agent 5 在 7999 次迭代中的资源申请量为: 57.4613625781287
agent 6 在 7999 次迭代中的资源申请量为: 55.24087110717772
agent 7 在 7999 次迭代中的资源申请量为: 65.50028854905379
agent 8 在 7999 次迭代中的资源申请量为: 46.24608275294365
agent 9 在 7999 次迭代中的资源申请量为: 65.52079979416222
agent 10 在 7999 次迭代中的资源申请量为: 54.13915206740807
agent 11 在 7999 次迭代中的资源申请量为: 65.32780791909349
agent 12 在 7999 次迭代中的资源申请量为: 38.33588246360446
agent 13 在 7999 次迭代中的资源申请量为: 49.60715638022637
agent 14 在 7999 次迭代中的资源申请量为: 41.29091938244435
agent 15 在 7999 次迭代中的资源申请量为: 42.41111668392344
agent 16 在 7999 次迭代中的资源申请量为: 49.15505086967045
agent 17 在 7999 次迭代中的资源申请量为: 50.02743433376963
agent 18 在 7999 次迭代中的资源申请量为: 46.451833863474775
agent 19 在 7999 次迭代中的资源申请量为: 44.69036994575968
agent 20 在 7999 次迭代中的资源申请量为: 50.711114124096575
agent 21 在 7999 次迭代中的资源申请量为: 69.80435251127331
agent 

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.16840936604421
agent 2 在 9999 次迭代中的资源申请量为: 56.00853994372033
agent 3 在 9999 次迭代中的资源申请量为: 60.6146548464284
agent 4 在 9999 次迭代中的资源申请量为: 37.912752865983734
agent 5 在 9999 次迭代中的资源申请量为: 57.5358638913341
agent 6 在 9999 次迭代中的资源申请量为: 55.312325361540765
agent 7 在 9999 次迭代中的资源申请量为: 65.58734664246651
agent 8 在 9999 次迭代中的资源申请量为: 46.30596926351353
agent 9 在 9999 次迭代中的资源申请量为: 65.60681324960835
agent 10 在 9999 次迭代中的资源申请量为: 54.20971336236532
agent 11 在 9999 次迭代中的资源申请量为: 65.41547828149595
agent 12 在 9999 次迭代中的资源申请量为: 38.38551966693003
agent 13 在 9999 次迭代中的资源申请量为: 49.67308829142777
agent 14 在 9999 次迭代中的资源申请量为: 41.34389535803232
agent 15 在 9999 次迭代中的资源申请量为: 42.465752086092806
agent 16 在 9999 次迭代中的资源申请量为: 49.21873852018184
agent 17 在 9999 次迭代中的资源申请量为: 50.091716338870555
agent 18 在 9999 次迭代中的资源申请量为: 46.51214350357571
agent 19 在 9999 次迭代中的资源申请量为: 44.749330375808576
agent 20 在 9999 次迭代中的资源申请量为: 50.77623576057697
agent 21 在 9999 次迭代中的资源申请量为: 69.89575471410086
agent

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.19880558169281
agent 2 在 51999 次迭代中的资源申请量为: 56.038712039744865
agent 3 在 51999 次迭代中的资源申请量为: 60.64796214438809
agent 4 在 51999 次迭代中的资源申请量为: 37.93333499800165
agent 5 在 51999 次迭代中的资源申请量为: 57.566936866986715
agent 6 在 51999 次迭代中的资源申请量为: 55.342127471202595
agent 7 在 51999 次迭代中的资源申请量为: 65.62365679460461
agent 8 在 51999 次迭代中的资源申请量为: 46.330946703571904
agent 9 在 51999 次迭代中的资源申请量为: 65.64268770459287
agent 10 在 51999 次迭代中的资源申请量为: 54.239143036902455
agent 11 在 51999 次迭代中的资源申请量为: 65.4520437985208
agent 12 在 51999 次迭代中的资源申请量为: 38.40622233033338
agent 13 在 51999 次迭代中的资源申请量为: 49.70058714458009
agent 14 在 51999 次迭代中的资源申请量为: 41.36599055513415
agent 15 在 51999 次迭代中的资源申请量为: 42.48853939611912
agent 16 在 51999 次迭代中的资源申请量为: 49.24530133805132
agent 17 在 51999 次迭代中的资源申请量为: 50.118527049896485
agent 18 在 51999 次迭代中的资源申请量为: 46.537297422316804
agent 19 在 51999 次迭代中的资源申请量为: 44.77392156660176
agent 20 在 51999 次迭代中的资源申请量为: 50.80339666469733
agent 21 在 51999 次迭代中的资源申请量为

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.198805581714595
agent 2 在 53999 次迭代中的资源申请量为: 56.03871203976649
agent 3 在 53999 次迭代中的资源申请量为: 60.64796214441196
agent 4 在 53999 次迭代中的资源申请量为: 37.9333349980164
agent 5 在 53999 次迭代中的资源申请量为: 57.566936867008984
agent 6 在 53999 次迭代中的资源申请量为: 55.342127471223954
agent 7 在 53999 次迭代中的资源申请量为: 65.62365679463063
agent 8 在 53999 次迭代中的资源申请量为: 46.3309467035898
agent 9 在 53999 次迭代中的资源申请量为: 65.64268770461858
agent 10 在 53999 次迭代中的资源申请量为: 54.239143036923544
agent 11 在 53999 次迭代中的资源申请量为: 65.45204379854701
agent 12 在 53999 次迭代中的资源申请量为: 38.406222330348214
agent 13 在 53999 次迭代中的资源申请量为: 49.7005871445998
agent 14 在 53999 次迭代中的资源申请量为: 41.365990555149985
agent 15 在 53999 次迭代中的资源申请量为: 42.48853939613545
agent 16 在 53999 次迭代中的资源申请量为: 49.245301338070355
agent 17 在 53999 次迭代中的资源申请量为: 50.1185270499157
agent 18 在 53999 次迭代中的资源申请量为: 46.53729742233483
agent 19 在 53999 次迭代中的资源申请量为: 44.77392156661938
agent 20 在 53999 次迭代中的资源申请量为: 50.8033966647168
agent 21 在 53999 次迭代中的资源申请量为: 69

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.19880558172394
agent 2 在 55999 次迭代中的资源申请量为: 56.03871203977576
agent 3 在 55999 次迭代中的资源申请量为: 60.647962144422195
agent 4 在 55999 次迭代中的资源申请量为: 37.93333499802272
agent 5 在 55999 次迭代中的资源申请量为: 57.56693686701853
agent 6 在 55999 次迭代中的资源申请量为: 55.34212747123311
agent 7 在 55999 次迭代中的资源申请量为: 65.62365679464179
agent 8 在 55999 次迭代中的资源申请量为: 46.33094670359748
agent 9 在 55999 次迭代中的资源申请量为: 65.64268770462961
agent 10 在 55999 次迭代中的资源申请量为: 54.239143036932596
agent 11 在 55999 次迭代中的资源申请量为: 65.45204379855825
agent 12 在 55999 次迭代中的资源申请量为: 38.40622233035458
agent 13 在 55999 次迭代中的资源申请量为: 49.70058714460825
agent 14 在 55999 次迭代中的资源申请量为: 41.36599055515678
agent 15 在 55999 次迭代中的资源申请量为: 42.48853939614246
agent 16 在 55999 次迭代中的资源申请量为: 49.24530133807852
agent 17 在 55999 次迭代中的资源申请量为: 50.11852704992394
agent 18 在 55999 次迭代中的资源申请量为: 46.53729742234256
agent 19 在 55999 次迭代中的资源申请量为: 44.77392156662694
agent 20 在 55999 次迭代中的资源申请量为: 50.80339666472515
agent 21 在 55999 次迭代中的资源申请量为: 69

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.198805581727676
agent 2 在 57999 次迭代中的资源申请量为: 56.03871203977947
agent 3 在 57999 次迭代中的资源申请量为: 60.647962144426295
agent 4 在 57999 次迭代中的资源申请量为: 37.93333499802526
agent 5 在 57999 次迭代中的资源申请量为: 57.566936867022356
agent 6 在 57999 次迭代中的资源申请量为: 55.34212747123678
agent 7 在 57999 次迭代中的资源申请量为: 65.62365679464625
agent 8 在 57999 次迭代中的资源申请量为: 46.33094670360055
agent 9 在 57999 次迭代中的资源申请量为: 65.64268770463403
agent 10 在 57999 次迭代中的资源申请量为: 54.239143036936206
agent 11 在 57999 次迭代中的资源申请量为: 65.45204379856276
agent 12 在 57999 次迭代中的资源申请量为: 38.406222330357124
agent 13 在 57999 次迭代中的资源申请量为: 49.70058714461163
agent 14 在 57999 次迭代中的资源申请量为: 41.36599055515949
agent 15 在 57999 次迭代中的资源申请量为: 42.48853939614526
agent 16 在 57999 次迭代中的资源申请量为: 49.24530133808179
agent 17 在 57999 次迭代中的资源申请量为: 50.11852704992724
agent 18 在 57999 次迭代中的资源申请量为: 46.53729742234566
agent 19 在 57999 次迭代中的资源申请量为: 44.773921566629966
agent 20 在 57999 次迭代中的资源申请量为: 50.80339666472849
agent 21 在 57999 次迭代中的资源申请量为

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.19880558172988
agent 2 在 59999 次迭代中的资源申请量为: 56.03871203978166
agent 3 在 59999 次迭代中的资源申请量为: 60.64796214442871
agent 4 在 59999 次迭代中的资源申请量为: 37.93333499802675
agent 5 在 59999 次迭代中的资源申请量为: 57.56693686702461
agent 6 在 59999 次迭代中的资源申请量为: 55.34212747123894
agent 7 在 59999 次迭代中的资源申请量为: 65.6236567946489
agent 8 在 59999 次迭代中的资源申请量为: 46.330946703602365
agent 9 在 59999 次迭代中的资源申请量为: 65.64268770463663
agent 10 在 59999 次迭代中的资源申请量为: 54.23914303693835
agent 11 在 59999 次迭代中的资源申请量为: 65.4520437985654
agent 12 在 59999 次迭代中的资源申请量为: 38.406222330358624
agent 13 在 59999 次迭代中的资源申请量为: 49.70058714461363
agent 14 在 59999 次迭代中的资源申请量为: 41.3659905551611
agent 15 在 59999 次迭代中的资源申请量为: 42.48853939614691
agent 16 在 59999 次迭代中的资源申请量为: 49.24530133808371
agent 17 在 59999 次迭代中的资源申请量为: 50.11852704992918
agent 18 在 59999 次迭代中的资源申请量为: 46.537297422347486
agent 19 在 59999 次迭代中的资源申请量为: 44.77392156663175
agent 20 在 59999 次迭代中的资源申请量为: 50.80339666473046
agent 21 在 59999 次迭代中的资源申请量为: 69.9

In [6]:
np.savetxt('../data/01-DGD-CTM-robust-agent_resource_quantity_small_value_deterministic_bias.txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/01-DGD-CTM-robust-sum_cost_small_value_deterministic_bias.txt',sum_cost,fmt='%0.8f')#用户成本总和
np.savetxt('../data/01-DGD-CTM-robust-sum_resource_small_value_deterministic_bias.txt',sum_resource,fmt='%0.8f')#用户资源总和